In [11]:
import numpy as np
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import neal

In [14]:
#リターンベクトル(縦ベクトルとして定義)
r=np.array([[0.01,0.02,0.03,0.04,0.05]]).T
#共分散行列
C=np.array([[ 0.01   , -0.0062 ,  0.0238 ,  0.0209 ,  0.0096 ],\
            [-0.0062 ,  0.0225 ,  0.01455,  0.0264 ,  0.0147 ],\
            [ 0.0238 ,  0.01455,  0.04   ,  0.0114 ,  0.0049 ],\
            [ 0.0209 ,  0.0264 ,  0.0114 ,  0.0625 ,  0.0057 ],\
            [ 0.0096 ,  0.0147 ,  0.0049 ,  0.0057 ,  0.09   ]])

#N=5資産D=10量子ビットを各資産に対応
N=5
D=10
K=6

#ガンマとラムダを定義
gamma=1.2
lam=0.2

In [15]:
#QUBO matrix を計算

#変換の為の配列を作成

#変換行列B(表記はわかりにくですね..)
B=np.vstack([np.roll([(1 if i<D else 0) for i in range(N*D)],j*D) for j in range(N)])

#要素が全て1のベクトル
l=np.ones(N)

A=np.ones((N,N))

Q=-np.diag(np.dot(r.T,B))+gamma*np.dot(B.T,np.dot(C,B))+lam*(np.dot(B.T,np.dot(A,B))-2*K*np.diag(np.dot(l,B)))

Q=np.triu(Q)+np.triu(Q,k=1) #非対角成分を二倍し上三角行列に
Q=Q/np.max(np.abs(Q))  #規格化しないと桁落ちにより正しい解が得られないことがあります


In [16]:
#問題を与える時は辞書にして渡す必要があるので変換

Qdict=dict()

for i in range(N*D):
    for j in range(N*D):
        if i<=j:
            Qdict.update({(i,j):Q[i,j]})

In [7]:
#問題をapiに投げ、結果を取得 (*)
# sampler = EmbeddingComposite(DWaveSampler()) 
# response = sampler.sample_qubo(Qdict,num_reads=10000)

In [8]:
#目的関数が最小な解を取得し、ウェイトに変換
answer=response.lowest().record[0][0]
weight=np.dot(B,answer)/D
print(weight)

[0.1 0.1 0.  0.1 0. ]


In [17]:
#問題をapiに投げ、結果を取得 (*)
sampler = neal.SimulatedAnnealingSampler() 
response = sampler.sample_qubo(Qdict,num_reads=10000)

In [18]:
answer=response.lowest().record[0][0]
weight=np.dot(B,answer)/D
print(weight)

[0.4 0.2 0.  0.  0. ]
